In [1]:
import numpy as np
import paddle
import paddle.fluid as fluid
import random
import gym
from collections import deque

In [2]:
num_episodes = 500
num_exploration_episodes = 100
max_len_episode = 1000
batch_size = 32
learning_rate = 1e-3
gamma = 1.
initial_epsilon = 1.
final_epsilon = 0.01

In [3]:
def QNetWork(ipt):
    fc1 = fluid.layers.fc(input=ipt, size=24, act='relu')
    fc2 = fluid.layers.fc(input=fc1, size=24, act='relu')
    out = fluid.layers.fc(input=fc2, size=2)
    return out

In [4]:
state = fluid.layers.data(name='state', shape=[4], dtype='float32')
action = fluid.layers.data(name='action', shape=[1], dtype='int64')
reward = fluid.layers.data(name='reward', shape=[], dtype='float32')
next_state = fluid.layers.data(name='next_state', shape=[4], dtype='float32')
done = fluid.layers.data(name='done', shape=[], dtype='bool')

In [5]:
model = QNetWork(state)

In [6]:
predict_program = fluid.default_main_program().clone()

reward = fluid.layers.clip(reward, min=-1.0, max=1.0)

action_onehot = fluid.layers.one_hot(action, 4)
action_onehot = fluid.layers.cast(action_onehot, dtype='float32')

pred_action_value = fluid.layers.reduce_sum(fluid.layers.elementwise_mul(action_onehot, model), dim=1)

targetQ_predict_value = QNetWork(next_state)
best_v = fluid.layers.reduce_max(targetQ_predict_value, dim=1)
best_v.stop_gradient = True

target = reward + (1.0 - fluid.layers.cast(done, dtype='float32')) * 0.99 * best_v
cost = fluid.layers.square_error_cost(pred_action_value, target)
avg_cost = fluid.layers.reduce_mean(cost)

In [7]:
optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.002)
opt = optimizer.minimize(avg_cost)

In [8]:
place = fluid.CPUPlace()
# place = fluid.CUDAPlace(0)
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

[]

In [9]:
env = gym.make("CartPole-v1")

In [12]:
replay_buffer = deque(maxlen=10000)
for epsilon_id in range(num_episodes):
    state = env.reset()
    for t in range(100):
#         env.render()
        if random.random() < 0.01:
            action = env.action_space.sample()
        else:
            action = exe.run(predict_program,
                             feed={'state': np.array([state]).astype('float32')},
                             fetch_list=[model])
            action = np.squeeze(action, axis=0)
            action = np.argmax(action)

#         print(np.array([state]).astype('float32').shape)
        next_state, reward, done, info = env.step(action)
#         print(next_state, reward, done, info)

        reward = -10 if done else reward
        replay_buffer.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            break
        
        if len(replay_buffer) >= batch_size:
            batch_state, batch_action, batch_reward, batch_next_state, batch_done = \
                [np.array(a, dtype=np.float32) for a in zip(*random.sample(replay_buffer, batch_size))]
            
            print(batch_state.shape)
            print(batch_action.shape)
            print(batch_reward.shape)
            print(batch_next_state.shape)
            print(batch_done.shape)
            
            exe.run(fluid.default_main_program(),
                    feed={
                         'state': batch_state.astype('float32'),
                         'action': batch_action.astype('int64'),
                         'reward': batch_reward.astype('float32'),
                         'next_state': batch_next_state.astype('float32'),
                         'done': batch_done.astype('bool')})


(32, 4)
(32,)
(32,)
(32, 4)
(32,)


EnforceNotMet: Enforce failed. Expected x_dims.size() >= 2, but received x_dims.size():1 < 2:2.
Rank of Input(X) should be at least 2. at [/paddle/paddle/fluid/operators/one_hot_op.cc:32]
PaddlePaddle Call Stacks: 
0       0x7f5802ae86b6p paddle::platform::EnforceNotMet::EnforceNotMet(std::__exception_ptr::exception_ptr, char const*, int) + 486
1       0x7f5802e5a727p paddle::operators::OneHotOp::InferShape(paddle::framework::InferShapeContext*) const + 2759
2       0x7f580366514dp paddle::framework::OperatorWithKernel::RunImpl(paddle::framework::Scope const&, boost::variant<paddle::platform::CUDAPlace, paddle::platform::CPUPlace, paddle::platform::CUDAPinnedPlace, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::variant::void_, boost::detail::v